# DataVis Tutorial: Mapping with Geopandas

This notebook aims to provide an introduction to producing graphical maps of spatial data using shapefiles and the [`geopandas`](http://geopandas.org/mapping.html) package

### Set up environment

#### Set up geopandas

In [ ]:
import geopandas

We use a special version of geopandas with vectorized operations written in C, which provides very significant speedups over the current production version. We strongly recommend using the version of geopandas installed by default on [compute.rhg.com](https://compute.rhg.com), or installing using the dev branch from `conda-forge`:

```bash
conda install --channel conda-forge/label/dev geopandas
```

In [ ]:
geopandas.__version__

if you install the package this way, you should see something like `'1.0.0.dev+141.ge925363'` in the output above

#### Set up your notebook for displaying inline plots

In [ ]:
% matplotlib inline

## Using shapefiles

Shapefiles are a special ESRI file format for encoding spatial information such as geographic shapes (countries, states, oceans, etc), individual lines (borders, coastlines, rivers, etc), and points.

Shapefiles can be obtained from a number of sources. Here are some we use commonly:
* [US Census county boundaries](https://www.census.gov/geo/maps-data/data/cbf/cbf_counties.html)
* [US Census metropolitan and micropolitan statistical areas](https://www.census.gov/geo/maps-data/data/cbf/cbf_msa.html)
* [Natural Earth Global admin 0 regions (countries)](https://www.naturalearthdata.com/downloads/50m-cultural-vectors/)
* [Natural Earth Global admin_1 regions (states/provinces)](https://www.naturalearthdata.com/downloads/50m-cultural-vectors/)
* [Natural Earth physical features (land, ocean, rivers, lakes)](https://www.naturalearthdata.com/downloads/10m-physical-vectors)
* [US Department of Homeland Security: Electric Retail Service Territories](https://hifld-geoplatform.opendata.arcgis.com/datasets/c4fd0b01c2544a2f83440dab292f0980_0)
* [US Department of Homeland Security: FERC Regions](https://hifld-geoplatform.opendata.arcgis.com/datasets/ae9bc057efa44f23bde91a2cb8e996db_0)
* [US Department of Homeland Security: ISOs](https://hifld-geoplatform.opendata.arcgis.com/datasets/9d1099b016e5482c900d657f06f3ac80_0)

### Shapefiles on compute.rhg.com

A number of shapefiles can be found in the following directory:

```bash
/gcs/rhg-data/impactlab-rhg/spatial/shapefiles
```

### Reading in a shapefile

to read in a shapefile using geopandas, use the `read_file` method:

In [ ]:
shapefile = geopandas.read_file(
    '/gcs/rhg-data/impactlab-rhg/spatial/shapefiles/source/us_census/cbf_counties')

Print out the head of the file to view a sample of the contents:

In [ ]:
shapefile.head()

Shapefiles are usually structured in this way, where there are a number of data and metadata columns, along with a `'geometry'` column. This is a special designation in `geopandas` that allows the column's contents to be used in plotting.

### Plotting a column from the shapefile

In [ ]:
shapefile.plot('GEOID')

To limit this to a specific bounding box, set the axes limits using regular matplotlib syntax:

In [ ]:
ax = shapefile.plot('GEOID', figsize=(14, 8))
ax.set_xlim(-130, -65)
ax.set_ylim(22, 50)

### Modifying the data

the GEOID column is a string representation of FIPS, so isn't handled quite right. let's create a FIPS column:

In [ ]:
shapefile['FIPS'] = shapefile.GEOID.astype(int)

In [ ]:
ax = shapefile.plot('FIPS', figsize=(14, 8))
ax.set_xlim(-130, -65)
ax.set_ylim(22, 50)

In [ ]:
ax = shapefile.loc[shapefile.STATEFP == '06'].plot('FIPS', figsize=(14, 8))
# ax.set_xlim(-130, -65)
# ax.set_ylim(22, 50)

## Plotting data from another source on a shapefile

If you have data in another format, plotting it is as simple as adding it to the GeoDataFrame and plotting that column

In [ ]:
! head -n 5 /gcs/rhg-data/impactlab-rhg/acp/sector_data/ouptuts/cotton_output.csv

In [ ]:
import pandas as pd

In [ ]:
cotton_output = pd.read_csv(
    '/gcs/rhg-data/impactlab-rhg/acp/sector_data/ouptuts/cotton_output.csv',
    index_col=0)

In [ ]:
cotton_output.head()

In [ ]:
len(cotton_output)

It will be easier to join these dataframes if they are both indexed by FIPS:

In [ ]:
shapefile = shapefile.set_index('FIPS').sort_index()

In [ ]:
shapefile.head()

In [ ]:
centroids = shapefile.geometry.centroid

In [ ]:
shapefile['cotton_output'] = cotton_output['Value (Cotton Output - million dollars)']

In [ ]:
shapefile.head()

In [ ]:
type(shapefile)

In [ ]:
shapefile['cotton_output'] = shapefile['cotton_output'].where(shapefile['cotton_output'] > 0)

In [ ]:
ax = shapefile.plot('cotton_output', figsize=(14, 8), cmap='Greens')
ax.set_xlim(-130, -65)
ax.set_ylim(22, 50)

Let's clean this up a bit and set the N/A values to grey

In [ ]:
ax = shapefile.plot(figsize=(14, 8), color='lightgrey')
shapefile.loc[shapefile.cotton_output.notnull()].plot('cotton_output', figsize=(14, 8), cmap='Greens', ax=ax)
ax.set_xlim(-130, -65)
ax.set_ylim(22, 50)

### Combining figures with plots from other libraries

There are a number of other libraries that can manipulate plots in python, and luckily, many of them can work with matplotlib figures/axes objects. You can use them directly with matplotlib by plotting on the same axis:

#### Cartopy

[Cartopy](https://scitools.org.uk/cartopy/docs/v0.14/) is another library with out-of-the-box tools for plotting geographic information

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

The `coastlines` method can be called on any axis object that has been created with a cartopy projection.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

This can be combined with geopandas code to plot cartopy features on top of geopandas

In [ ]:
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax.coastlines('10m')

shapefile.plot(color='lightgrey', ax=ax)
shapefile.loc[shapefile.cotton_output.notnull()].plot('cotton_output', figsize=(14, 8), cmap='Greens', ax=ax)
ax.set_xlim(-130, -65)
ax.set_ylim(22, 50)

### Add custom lines

In [ ]:
ax = shapefile.plot(figsize=(14, 8), color='lightgrey')
shapefile.loc[shapefile.cotton_output.notnull()].plot('cotton_output', figsize=(14, 8), cmap='Greens', ax=ax)
ax.set_xlim(-130, -65)
ax.set_ylim(22, 50)

# by default, ax.plot will create a line plot
ax.plot(
    [-120, -70, -120],  # x values of line
    [25, 45, 45],  # y values of line
    color='blue',  # color of line
    linestyle='dotted')  # line style (optional)

ax.set_title('my plot')

### Bubble plot

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

ax.scatter(
    centroids.x, # x position of data
    centroids.y, # y position of data
    s=shapefile.cotton_output, # data to use for size
    c=shapefile.cotton_output, # data to use for coloring
    cmap='Greens') # color scheme - see https://matplotlib.org/examples/color/colormaps_reference.html

ax.coastlines('10m')